In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [5]:
from langchain_chroma import Chroma
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings

In [6]:
client = chromadb.PersistentClient(path='./chroma_db')
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
chroma_db = Chroma(client=client, collection_name="langchain",embedding_function=embeddings) 

In [7]:
retriever = chroma_db.as_retriever()

In [8]:
model_local = ChatOllama(model="llama3.2")

C:\Users\andre\AppData\Local\Temp\ipykernel_22936\2362477809.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model_local = ChatOllama(model="llama3.2")


In [14]:
# print("\n########\nAfter RAG\n")
after_rag_template = """You are a Medical Chatbot answering questions strictly related to chest diseases. Do not provide any consultation. Give answers only for medical queries. Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("tell me more about tuberculosis in 35 words?"))

Tuberculosis (TB) is a bacterial infection primarily affecting the lungs. Symptoms include cough, fever, and fatigue. TB can be treated with antibiotics, but resistance and relapse are concerns. Vaccination and early detection help prevent transmission.


## Saving to pickle

In [13]:
import pickle

In [15]:
with open("embedding.pkl", "wb") as f:
    pickle.dump(embeddings, f)

In [21]:
with open("embedding.pkl", "rb") as f:
    loaded_embeddings = pickle.load(f)
# loaded_embeddings.embed_query("what is effusion")

NameError: name 'loaded_embeddings' is not defined